# Title
The title of the notebook should be coherent with file name. Namely, file name should be:    
*author's initials_progressive number_title.ipynb*    
For example:    
*EF_01_Data Exploration.ipynb*

## Purpose
State the purpose of the notebook.

## Methodology
Quickly describe assumptions and processing steps.

## WIP - improvements
Use this section only if the notebook is not final.

Notable TODOs:
- todo 1;
- todo 2;
- todo 3.

## Results
Describe and comment the most important results.

## Suggested next steps
State suggested next steps, based on results obtained in this notebook.

# Setup

## Library import
We import all the required Python libraries

In [98]:
# Data manipulation
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta

num_users = 5000
fake = Faker()
unique_city_names = [fake.city() for _ in range(250)]
unique_state_names = [fake.state() for _ in range(250)]
city_to_state_map = dict(zip(unique_city_names, unique_state_names))

# Function to sanitize text fields by removing commas
def sanitize_text(text):
    return text.replace(',', '')

# Generate user data
user_data = pd.DataFrame({
    'userID': [i for i in range(1, num_users + 1)],
    'userName': [sanitize_text(fake.user_name()) for _ in range(num_users)],
    'email': [sanitize_text(fake.email()) for _ in range(num_users)],
    'password': [sanitize_text(fake.password()) for _ in range(num_users)],
    'signUpDate': [fake.date_between(start_date='-2y', end_date='today') for _ in range(num_users)],
    'dateOfBirth': [fake.date_of_birth(minimum_age=18, maximum_age=70) for _ in range(num_users)],
    'gender': [random.choice(['Male', 'Female', 'Other']) for _ in range(num_users)],
    'city': [random.choice(unique_city_names) for _ in range(num_users)]
})

user_data['state'] = user_data['city'].map(city_to_state_map)
user_data['membershipType'] = [random.choice(['Free', 'Premium']) for _ in range(num_users)]

def generate_credit_score():
    return min(max(int(np.random.normal(700, 50)), 300), 850)
user_data['CreditScore'] = [generate_credit_score() for _ in range(num_users)]



account_types = {
    'Checking': (100, 5000),  # Checking accounts usually have moderate balances
    'Savings': (500, 20000),  # Savings accounts might have higher balances
    'Credit Card': (-5000, -100),  # Credit card debt
    'Mortgage': (-250000, -50000),  # Mortgage values can be quite high
    'Car Loan': (-40000, -5000),  # Car loans vary depending on the vehicle
    'Investments': (1000, 100000),  # Investment accounts can have a wide range
    'Student Loan': (-50000, -1000)  # Student loans can vary widely
}
unique_bank_names = [fake.company() for _ in range(50)]

# Modify the accounts_data creation
accounts_data = []
account_id = 1
for user_id in range(1, num_users + 1):
    for account_type in ['Checking', 'Savings']:
        balance_range = account_types[account_type]
        balance = round(random.uniform(*balance_range), 2)
        bank_name = random.choice(unique_bank_names)
        account = {
            'accountID': account_id,
            'userID': user_id,
            'accountType': account_type,
            'accountName': bank_name,  # AccountName is now just the BankName
            'bankName': bank_name,
            'currentBalance': balance
        }
        accounts_data.append(account)
        account_id += 1

    # Add 0-1 additional random accounts per user
    num_additional_accounts = random.randint(1, 5)
    for _ in range(num_additional_accounts):
        account_type, balance_range = random.choice(list(account_types.items()))
        # Skip if it's a Checking or Savings account since we already added those
        if account_type in ['Checking', 'Savings']:
            continue

        balance = round(random.uniform(*balance_range), 2)
        account_name = random.choice(account_names)  # Choose from the pre-generated list

        account = {
            'accountID': account_id,
            'userID': user_id,
            'accountType': account_type,
            'accountName': account_name,
            'bankName': fake.company(),
            'currentBalance': balance
        }
        accounts_data.append(account)
        account_id += 1
accounts_data = pd.DataFrame(accounts_data)

num_accounts = account_id - 1  # account_id from the last used value in the Accounts table script

transaction_categories = ['Groceries', 'Entertainment', 'Bills', 'Dining', 'Shopping', 'Travel', 'Income']
unique_company_names = [fake.company() for _ in range(100)]
transactions_data = []
for account in accounts_data.itertuples():
    user_sign_up_date = user_data.loc[account.userID - 1, 'signUpDate']
    num_transactions = random.randint(5, 20)
    for _ in range(num_transactions):
        category = random.choice(transaction_categories)
        transaction_id = random.randint(100000, 999999)  # 6-digit random number
        date = fake.date_between(start_date=user_sign_up_date, end_date='today')
        if category == 'Income':
            amount = round(random.uniform(1000, 5000), 2)  # Larger amount for Income
            transaction_type = 'Debit'
        else:
            amount = round(random.uniform(5, 500), 2)
            transaction_type = 'Credit'
        transaction = {
            'transactionID': transaction_id,
            'accountID': account.accountID,
            'Date': date,
            'name': random.choice(unique_company_names),
            'amount': amount,
            'category': category,
            'transactionType': transaction_type
        }
        transactions_data.append(transaction)
transactions_data = pd.DataFrame(transactions_data)


login_history_data = []
for user_id in range(1, num_users + 1):
    user_sign_up_date = user_data.loc[user_id - 1, 'signUpDate']
    num_logins = random.randint(1, 30)  # Random number of logins for each user

    for _ in range(num_logins):
        login_id = random.randint(10000, 99999)  
        login_time = fake.date_time_between(start_date=user_sign_up_date, end_date='now')
        logout_time = login_time + timedelta(hours=random.uniform(0.1, 1))  # Logout within 1 hour
        device_type = random.choices(['Mobile', 'Desktop'], weights=[80, 20], k=1)[0]
        ip_address = fake.ipv4()

        login_history = {
            'loginID': login_id,
            'userID': user_id,
            'loginTime': login_time.strftime("%Y-%m-%d %H:%M:%S"),
            'logoutTime': logout_time.strftime("%Y-%m-%d %H:%M:%S"),
            'deviceType': device_type,
            'IPAddress': ip_address
        }
        login_history_data.append(login_history)

login_history_data = pd.DataFrame(login_history_data)


# ... [Your existing user_data, accounts_data, transactions_data, login_history_data generation code] ...

# Define some additional creative ActivityTypes
additional_activity_types = [
    'Reviewed Transaction History','Checked Credit Score', 'View Budget', 'View Goals', 'View Accounts', 'Viewed Notifications'
]

# Combine your provided ActivityTypes with the additional ones
activity_types = ['Created Budget', 'Set Goal', 'Turned off notifications', 'Added Account'] + additional_activity_types

# Create User Activity data
activity_data = []
activity_id = 1000000  # Starting from a 7-digit number

for login in login_history_data.itertuples():
    num_activities = random.randint(2, 5)  # Random number of activities per login
    login_time_obj = datetime.strptime(login.loginTime, "%Y-%m-%d %H:%M:%S")  # Convert back to datetime
    logout_time_obj = datetime.strptime(login.logoutTime, "%Y-%m-%d %H:%M:%S")  # Convert logoutTime to datetime

    for _ in range(num_activities):
        # Use the datetime object for generating activity time
        activity_time = fake.date_time_between(start_date=login_time_obj, end_date=logout_time_obj)
        activity = {
            'activityID': activity_id,
            'userID': login.userID,
            'loginID': login.loginID,
            'activityType': random.choice(activity_types),
            'activityTime': activity_time.strftime("%Y-%m-%d %H:%M:%S")
        }
        activity_data.append(activity)
        activity_id += 1

activity_data = pd.DataFrame(activity_data)

accounts_data['accountName'] = accounts_data['accountName'].apply(sanitize_text)
accounts_data['bankName'] = accounts_data['bankName'].apply(sanitize_text)
transactions_data['name'] = transactions_data['name'].apply(sanitize_text)

#Save to CSV
base_directory = r"C:\Users\19043\Documents\saas tables"
user_data.to_csv(f"{base_directory}\\user_data.csv", index=False)
accounts_data.to_csv(f"{base_directory}\\account_data.csv", index=False)
transactions_data.to_csv(f"{base_directory}\\transactions_data.csv", index=False)
login_history_data.to_csv(f"{base_directory}\\login_history_data.csv", index=False)
activity_data.to_csv(f"{base_directory}\\activity_data.csv", index=False)

In [99]:
user_data

,userID,userName,email,password,signUpDate,dateOfBirth,gender,city,state,membershipType,CreditScore
0,1,hamiltoncarla,karenhernandez@example.com,h09aFhIv(5,2022-09-24,1964-02-11,Female,Laurenshire,New Jersey,Free,679
1,2,stewartwhitney,chenkristi@example.net,F$879Nxilp,2023-05-25,1993-04-21,Other,North Kiaraborough,Montana,Free,770
2,3,amandagonzales,qwright@example.net,X$JMSj@q(1,2023-10-06,1956-10-16,Female,Lake Robertville,Oregon,Free,687
3,4,ifernandez,miguelabbott@example.org,8d83I42l*5,2022-12-14,2000-02-22,Female,West Mathew,Kansas,Free,635
4,5,mdrake,iford@example.com,%zJKbWYb6^,2022-03-15,2003-07-24,Female,South Isaiahport,Vermont,Premium,774
...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,richardbarton,julie50@example.com,zlXV&Dyd&4,2022-08-03,1989-11-04,Female,South Brenda,West Virginia,Premium,720
4996,4997,jared44,yfowler@example.net,7pd9VkSA%Z,2023-08-01,1962-04-05,Other,Port Christopher,Alaska,Premium,661
4997,4998,uhenderson,salazarkelly@example.net,$zHwZ%vRd8,2023-01-08,1965-12-15,Other,North Mary,North Dakota,Free,655
4998,4999,gsolomon,wendy20@example.org,^6WSgx#Z9+,2022-05-10,2004-05-17,Male,New Laurenside,Vermont,Free,648


In [100]:
accounts_data

,accountID,userID,accountType,accountName,bankName,currentBalance
0,1,1,Checking,Phillips-Terry,Phillips-Terry,286.27
1,2,1,Savings,Chavez Schwartz and Williams,Chavez Schwartz and Williams,2152.30
2,3,1,Credit Card,Haynes-Jones Savings,Dennis Rodriguez and Pittman,-3921.42
3,4,1,Investments,Dean LLC Checking,Mcgee Rodriguez and Baxter,74953.90
4,5,1,Credit Card,Mitchell and Sons Mortgage,Mcgee-Johnson,-512.27
...,...,...,...,...,...,...
20843,20844,5000,Checking,Greer Cole and Garcia,Greer Cole and Garcia,1119.98
20844,20845,5000,Savings,Crawford Martin and Nelson,Crawford Martin and Nelson,13027.53
20845,20846,5000,Credit Card,Silva-Robertson Credit Card,Mclaughlin LLC,-1515.91
20846,20847,5000,Student Loan,Sutton LLC Checking,Garcia Ltd,-13704.11


In [101]:
transactions_data = pd.DataFrame(transactions_data)

transactions_data

,transactionID,accountID,Date,name,amount,category,transactionType
0,973768,1,2023-10-07,Hudson Wall and Parks,399.36,Entertainment,Credit
1,837962,1,2023-06-08,Sullivan Inc,1469.06,Income,Debit
2,483406,1,2023-08-01,Brown-Walker,413.48,Dining,Credit
3,356746,1,2023-06-16,Davidson-Jones,4227.38,Income,Debit
4,317833,1,2023-04-19,Mccormick Anderson and Dyer,260.92,Shopping,Credit
...,...,...,...,...,...,...,...
260226,601116,20848,2023-05-23,Cook Inc,130.36,Entertainment,Credit
260227,240148,20848,2023-04-22,Watkins-Ortiz,165.48,Entertainment,Credit
260228,291143,20848,2022-09-04,Lowery-Torres,225.21,Travel,Credit
260229,376683,20848,2022-09-08,Hurley-Sanders,1093.37,Income,Debit


In [102]:
login_history_data

,loginID,userID,loginTime,logoutTime,deviceType,IPAddress
0,13591,1,2023-11-16 00:55:46,2023-11-16 01:13:48,Mobile,222.133.46.82
1,31132,1,2022-11-21 06:22:59,2022-11-21 07:15:59,Mobile,181.222.99.65
2,40820,1,2022-11-26 16:12:52,2022-11-26 16:38:08,Mobile,24.99.148.214
3,74679,1,2023-03-03 21:14:29,2023-03-03 21:48:24,Mobile,144.136.14.28
4,52294,1,2023-05-20 00:52:57,2023-05-20 01:23:40,Mobile,36.204.39.106
...,...,...,...,...,...,...
78457,15820,5000,2023-09-03 04:06:55,2023-09-03 04:34:02,Desktop,200.213.137.86
78458,81365,5000,2023-10-21 00:56:56,2023-10-21 01:31:49,Desktop,161.160.132.216
78459,12173,5000,2022-12-18 00:04:32,2022-12-18 00:50:46,Mobile,54.68.234.189
78460,74421,5000,2023-01-02 01:21:47,2023-01-02 01:52:16,Mobile,159.83.88.204


In [103]:
activity_data

,activityID,userID,loginID,activityType,activityTime
0,1000000,1,13591,View Accounts,2023-11-16 01:00:33
1,1000001,1,13591,Reviewed Transaction History,2023-11-16 01:02:02
2,1000002,1,31132,Added Account,2022-11-21 06:57:38
3,1000003,1,31132,View Budget,2022-11-21 06:26:29
4,1000004,1,31132,Created Budget,2022-11-21 07:09:58
...,...,...,...,...,...
274771,1274771,5000,74421,View Accounts,2023-01-02 01:40:19
274772,1274772,5000,74421,Added Account,2023-01-02 01:47:13
274773,1274773,5000,83622,Created Budget,2022-10-28 22:16:27
274774,1274774,5000,83622,Added Account,2022-10-28 22:17:20
